## Doing Transformations on All the Tables (Changing dateformats)

In [0]:
#creating a for loop which  will automate the work of reading the tables from the folder
table_name = []

for i in dbutils.fs.ls('mnt/bronze/SalesLT/'):
 table_name.append(i.name.split('/')[0])

In [0]:
table_name

['Address ',
 'Customer ',
 'CustomerAddress ',
 'Product ',
 'ProductCategory ',
 'ProductDescription ',
 'ProductModel ',
 'ProductModelProductDescription ',
 'SalesOrderDetail ',
 'SalesOrderHeader ']

In [0]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

# Here we are using 'i' to extract the table name from the list
for i in table_name:
    # Strip any extra spaces from 'i' to avoid path issues
    path = '/mnt/bronze/SalesLT/' + i.strip() + '/' + i.strip() + '.parquet'   #<<<<< Corrected path
    df = spark.read.format('parquet').load(path)
    columns = df.columns

    # Creating a loop to check if any of the columns have 'Date' or 'date' in it
    for col in columns:
        if "Date" in col or "date" in col:
            
            df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))

    # Writing the data to the silver table
    output_path = '/mnt/silver/SalesLT/' + i.strip() + '/'
    df.write.format('delta').mode('overwrite').save(output_path)  # Storing the data in Delta format
